<a href="https://colab.research.google.com/github/dinanabila/eda-notebooks/blob/main/2025/eda_submission_pemda_dicoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exploratory data analysis ini dilakukan sebagai panduan dalam merancang script transform untuk proses ETL submission kelas Dicoding Belajar Fundamental Pemrosesan Data.

Dataset yang digunakan diperoleh dari hasil scraping web berikut: https://fashion-studio.dicoding.dev

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/dinanabila/eda-notebooks/refs/heads/main/data/submission-pemda-dicoding.csv')
df.head()

,Title,Price,Rating,Colors,Size,Gender,Timestamp
0,Unknown Product,$100.00,Invalid Rating / 5,5,M,Men,2025-07-19T07:00:01.308790
1,T-shirt 2,$102.15,3.9 / 5,3,M,Women,2025-07-19T07:00:01.308790
2,Hoodie 3,$496.88,4.8 / 5,3,L,Unisex,2025-07-19T07:00:01.308790
3,Pants 4,$467.31,3.3 / 5,3,XL,Men,2025-07-19T07:00:01.308790
4,Outerwear 5,$321.59,3.5 / 5,3,XXL,Women,2025-07-19T07:00:01.308790


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      1000 non-null   object
 1   Price      1000 non-null   object
 2   Rating     1000 non-null   object
 3   Colors     1000 non-null   int64 
 4   Size       1000 non-null   object
 5   Gender     1000 non-null   object
 6   Timestamp  1000 non-null   object
dtypes: int64(1), object(6)
memory usage: 54.8+ KB


Berdasarkan info df di atas, dapat dilihat terdapat kolom yang tipe datanya masih belum sesuai: Price dan Rating. Kita ubah dulu. EITS tapi sebelum itu, kita hapus dulu simbol $ di setiap value kolom Price, soalnya tadi error karena harus pure angka kalau mau ubah tipe data ke float.

In [4]:
df['Price'] = df['Price'].str.replace('$', '')
df['Price'].head(2)

,Price
0,100.00
1,102.15


Daaann, sebelum beneran bisa diubah tipe datanya, kita harus tangani dulu value non angka yang masih ada di kolom Price. Aku bisa tau, soalnya dapat `ValueError: could not convert string to float: 'Price Unavailable'` pas nge-run kode buat ubah tipe data Price ke float.

Untuk penanganannya, kita hapus baris dengan Price yang bukan angka.

In [9]:
df = df[(df['Price'] != 'Price Unavailable') & (df['Price'].notna())].copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 967 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      967 non-null    object
 1   Price      967 non-null    object
 2   Rating     967 non-null    object
 3   Colors     967 non-null    int64 
 4   Size       967 non-null    object
 5   Gender     967 non-null    object
 6   Timestamp  967 non-null    object
dtypes: int64(1), object(6)
memory usage: 60.4+ KB


In [5]:
df['Price'] = df['Price'].astype('float64')
df['Rating'] = df['Rating'].astype('float64')
df.info()

ValueError: could not convert string to float: 'Price Unavailable'

# **Title**

In [ ]:
df.Title.nunique()

Hanya ada 901 data unik dari 1000 data? Berarti ada yang duplikat dong? Mari kita periksa.

In [ ]:
print(f"Jumlah duplikasi data: {df.duplicated().sum()}")

Ternyata ada 50 data duplikasi. Ini **harus kita hapus nanti**.

Btw tentang duplikasi, jadi kepikiran. Kira-kira ada ga ya nama produk yang muncul lebih dari sekali? Soalnya kan itu juga sama aja kek data duplikat? Coba kita periksa.

In [ ]:
df['Title'].value_counts()

Wih ada 100 Unknown Product. Banyak juga ya. Tapi tetap **harus kita hapus**, sebab tidak memberikan informasi tentang produk fashion apapun.

In [ ]:
# hapus data duplikat
df.drop_duplicates(inplace=True)

# hapus data dengan value kolom Title == 'Unknown Product'
df = df[df['Title'] != 'Unknown Product'].copy()

# cek df sekarang
df.info()

# **Price**